In [22]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

from scwidgets import (AnswerRegistry, TextareaAnswer, CodeDemo,
                       ParametersBox, PyplotOutput, ClearedOutput,
                       AnimationOutput,CheckRegistry)

from widget_code_input import WidgetCodeInput
from ipywidgets import Layout, Output, Textarea

from ase import Atoms

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Before you work on this module, please input your name to create a file that will store your answers, or load an existing file if you need to continue.

In [2]:
check_registry = CheckRegistry()  # this is needed to coordinate code checking
answer_registry = AnswerRegistry(prefix="module_xx")
display(answer_registry)

AnswerRegistry(children=(Output(layout=Layout(height='99%', width='99%')), HBox(children=(Dropdown(description…

# A brief Python re-cap

This course assumes basic knowledge of the Python language (variables, conditionals, lists, function definitions...). If you are not familiar with these concepts, or with the syntax that is used to manipulate them in Python, you should follow some online crash-course, e.g. **XXXXX give some links**.

In this module we focus on three concepts that some may not be familiar with, and that are used heavily in this class.

* Modules, and how to import functionalities from external packages
* Python objects: variables and methods, from a user perspective
* Numpy arrays: numerical lists for maths, indexing and operations

# Extending Python through modules

One of the features that made Python so successful is how easy it is to extend its capabilities by including external libraries into a program.  

<div style="text-align:center">
<img src="./figures/xkcd-python.png"/><br>
    <a href="https://xkcd.com/353/">Reproduced from xkcd.com</a>
    Licensed under <a href="https://creativecommons.org/licenses/by-nc/2.5/">CC-BY-NC-2.5</a>
</div>

There are several ways to include features from external packages, and it is important to understand what are the implications for _using_ those features. For example, let's consider the sine function `sin`. Python does not have built-in trigonometric functions, so we need to import them from and external library that implements the sine function, for example the `math` package.

The simplest way to include functionality from a package is to import the whole library, using the syntax 
```python 
import library as alias
``` 
The `as alias` part is optional and allows to provide a shorthand for when a library is used a lot or has a very long name. In order to access the functions provided by `library`, it is then necessary to specify the `alias`, too, e.g. `alias.function(1)`. If `alias` is not given, one simply uses `library.function(1)`. This is because the functions are not brought into the main namespace, but remain part of the library. For example,

```python
import math
print(math.sin(1.0))
```

It is also possible to import just a few selected functions, and bring them into the main namespace. This is useful when only a few of the functionalities offered by `library` need to be used a lot. The syntax is then `from library import function`, for example
```python
from math import sin
print(sin(1.0))
```
It is also prossible to bring in _all_ functions from library into the main namespace using `from library import *`, but this is discoruaged, as it increases the risk of having clashes between the names of functions imported from different packages. 

In [3]:
# initialize a widget for code input
ex01_wci = WidgetCodeInput(
        function_name = "sin_cos", 
        function_parameters="x",
        code_theme = "default",
        docstring="""
Computes the sine and the cosine of an input value, and return both as a tuple

:param x: the input angle, in radians

:return: sin(x), cos(x)
""",
            function_body="""
# write code to import the sine and cosine function from the `math` module


# compute sine and cosine
s = 0
c = 1

return s, c
"""
)

# initialize a CodeDemo object and initialize its checks in the check_registry:
ex01_demo = CodeDemo( 
            code_input = ex01_wci,
            check_registry = check_registry,
            ) 

check_registry.add_check(ex01_demo,
                         inputs_parameters=[{"x" : 0}, {"x":np.pi}, {"x": 0.12345} ],
                         reference_outputs=[(0.0, 1.0), (0, -1.0), (0.12313667785133202, 0.9923897211114882)],
                         equal=np.allclose)
# Registers the answer
answer_registry.register_answer_widget("ex01_function", ex01_demo)

In [4]:
# helper function to generate values for checking the function
check_registry.print_reference_outputs(ex01_demo, ignore_errors=True)

Check 0:
[(0, 1), (0, 1), (0, 1)]


<span style="color:blue"> **01** Write a function that imports the sine and cosine functions from the `math` library, and returns simultaneously sine and cosine of the argument. </span>

_NB: it is possible to import python packages also within the body of a function. You can try different ways of importing the functions._

In [5]:
display(ex01_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unchecked')), VBox(…

# Python objects from a user perspective

This section provides an operative guide to _using_ Python classes, assuming no formal training in object-oriented programming. If you want to learn more (e.g. if you want to see how you can define your own objects) you can follow some online tutorials, e.g. **add a few links here**.

An object - in Python and elsewhere - is a code construct that holds both data (_member variables_) and functionality (_methods_). Combining data and code makes it possible to conceptualize the relationship between the attributes of an item, and the operations that can be performed with (or on) it, and is usually thought to lead to code that is clearer, easier to understand and maintain. Objects in Python can be used like any other variable - passed as arguments to a function, and returned as the output of a function call. 

Consider for example an object which is meant to represent a triangle. The geometry of a triangle can be entirely defined by its three sides, so we can combine in the same structure three floating point numbers describing the sides, `a, b, c`.  It would make sense to ask to compute the area of the triangle, or to scale it by a constant factor. A typical use case would be as follows

```python
my_triangle = Triangle(a=3.0, b=4.0, c=5.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")
print(f"Now scaling the triangle...")
my_triangle.scale(factor=2.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")
```

## Initialization

Let's look at this code in some more detail. The first line corresponds to the _initialization_ of the `Triangle` object. `Triangle` is the name of the _class_ that defines the object (usually this will have been imported by a library), and by calling it with tree arguments, corresponding to the length of the triangle sides, we create an _instance_ of a triangle with the desired values. One can create many instances of a class, and each can be manipulated independently

```python
triangle_one = Triangle(a=3.0, b=4.0, c=2.0)
triangle_two = Triangle(a=2.0, b=2.0, c=4.0)
```

The initialization might also run some checks, making sure that the object is created in a consistent state.

Once an instance of a class is created, it is possible to interact with it in many ways. One can retrieve the values of the member variables, or directly set their values

```python
print(f"The triangle has sides a={triangle_one.a}, b={triangle_one.b}, c={triangle_one.c}")
triangle_one.c = 5.0
triangle_two.c = 2.0
```

Note that manually setting the values of some member variables might leave the object in an invalid state (unless the programmer of the class has put further checks in place). 

In [6]:
import math
class Triangle:
    def __init__(self, a, b, c):        
        self.a, self.b, self.c = a, b, c
        if a+b<c or b+c<a or a+c<b:
            raise ValueError(f"Cannot create a triangle with sides {(a,b,c)}, as they violate triangle inequality.")
    
    def area(self):
        # heron formula. it will fail if a,b,c don't fulfill triangle inequality
        s = (self.a+self.b+self.c)/2       
        return math.sqrt(s*(s-self.a)*(s-self.b)*(s-self.c))
    
    def scale(self, factor):
        self.a*=factor; self.b*=factor; self.c*=factor; 
# ugly but necessary to make Triangle available in the CodeInput 
import builtins
builtins.Triangle = Triangle

In [7]:
# initialize a widget for code input
ex02_wci = WidgetCodeInput(
        function_name = "triangle_playground", 
        function_parameters="",
        code_theme = "default",
        docstring="""
Create and return a valid Triangle object

:return: An valid `Triangle` object
""",
            function_body="""
# NB: the Triangle class is not a builtin object and is only available here for this specific exercise

# Create an instance of a Triangle and return it
my_triangle = 0

return my_triangle
"""
)


def ex02_viz(code_input, visualizers):
    with visualizers[0]:
        triangle = code_input.get_function_object()()
        print(f"The triangle has sides a={triangle.a}, b={triangle.b}, c={triangle.c}")
        if (triangle.a+triangle.b<triangle.c or 
            triangle.b+triangle.c<triangle.a or
            triangle.a+triangle.c<triangle.b):
            print("The triangle sides do not fulfill the triangle inequality!")
            
# initialize a CodeDemo object and initialize its checks in the check_registry:
ex02_demo = CodeDemo( 
            code_input = ex02_wci,
            visualizers=[ClearedOutput()],
            update_visualizers=ex02_viz
            ) 

            
answer_registry.register_answer_widget("ex02_function", ex02_demo)

In [8]:
# helper function to generate values for checking the function
check_registry.print_reference_outputs(ex02_demo, ignore_errors=True)

<span style="color:blue"> **02a** Write a function that creates and returns a valid `Triangle` object. </span>

_Feel free to experiment setting and checking the values of the instance variables before converging on a correct answer. You can see the outputs of any `print` statement you include by running the checks._

In [9]:
display(ex02_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--out-of-date')), VBo…

<div style="color:blue"> <b>02b</b> Perform the following "experiments" and comment on what you observe.

* Create a triangle with sides $a=2$, $b=3$, $c=8$. 
* Create a triangle with sides $a=2$, $b=2$, $c=2$, but then manually set `my_triangle.c=12` before returning it
</div>

In [10]:
ex02_comments = TextareaAnswer(value='Answer here', layout=Layout(width='99%'))

answer_registry.register_answer_widget("ex02_answer", ex02_comments)

display(ex02_comments)

TextareaAnswer(children=(Textarea(value='Answer here', layout=Layout(width='99%')), VBox(children=(Button(desc…

In [11]:
# initialize a widget for code input
ex03_wci = WidgetCodeInput(
        function_name = "equilateral_triangle", 
        function_parameters="side",
        code_theme = "default",
        docstring="""
Creates an equilateral triangle given its side

:param side: The side of the triangle

:return: An equilateral `Triangle` object
""",
            function_body="""
# NB: the Triangle class is not a builtin object and is only available here for this specific exercise

# Create ai instance of a Triangle and return it
my_triangle = 0

return my_triangle
"""
)

# initialize a CodeDemo object and initialize its checks in the check_registry:
ex03_demo = CodeDemo( 
            code_input = ex03_wci,
            check_registry = check_registry,
            ) 

def ex03_asserts(output, reference):
    assert isinstance(output, Triangle), "The function should return a `Triangle` object"
    assert (output.a==output.b and output.a==output.c), "The triangle is not equilateral"  
    assert (output.a==reference[0]), "The side of the triangle does not match the input"

check_registry.add_check(ex03_demo,
                         inputs_parameters=[{"side" : 1}, {"side": 2}],
                         fingerprint= lambda t: (t.a, t.b, t.c),
                         reference_outputs=[(1,1,1), (2,2,2)],
                         custom_asserts=ex03_asserts
                         )

answer_registry.register_answer_widget("ex03_function", ex03_demo)

In [12]:
# helper function to generate values for checking the function
check_registry.print_reference_outputs(ex03_demo, ignore_errors=True)

<span style="color:blue"> **03** Write a function that creates a `Triangle` object, initialized to be an equilateral triangle with three equal sides equal to the argument passed to the function. </span>

In [13]:
display(ex03_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unchecked')), VBox(…

## Member functions and methods

Let's now shift our attention to how an object can be manipulated using member functions and methods. Member functions are called as `my_instance.member(arguments)` and can read and/or modify any instance variable of `my_instance`. 

For example, `my_triangle.area()` will return the area of the `Triangle` instance `my_triangle`, and `my_triangle.scale(2.0)` will double the size of all sides of `my_triangle`. 

In [14]:
# initialize a widget for code input
ex04_wci = WidgetCodeInput(
        function_name = "scaled_area", 
        function_parameters="triangle, scale_factor",
        code_theme = "default",
        docstring="""
Doubles the size of a triangle and returns its area.

:param triangle: A `Triangle` object, already initialized
:param scale_factor: How much the triangle sides should be scaled before computing the area

:return: The area of the triangle, after having its sides doubled in length
""",
            function_body="""
# scale up the sides of triangle

# compute its area
my_area = 0

return my_area
"""
)

ex04_demo = CodeDemo( 
            code_input = ex04_wci,
            check_registry = check_registry,
            ) 

check_registry.add_check(ex04_demo,
                         inputs_parameters=[{"triangle" : Triangle(3,4,5), 
                                             "scale_factor": 1}, 
                                            {"triangle" : Triangle(5,4,3), 
                                             "scale_factor": 2},],
                         reference_outputs=[6., 24.],
                         equal=np.allclose
                         )


            
answer_registry.register_answer_widget("ex04_function", ex04_demo)

<span style="color:blue"> **04** Write a function that takes a Triangle as input, doubles its size and returns the area of the scaled up triangle. </span>

In [15]:
display(ex04_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unchecked')), VBox(…

# NumPy arrays 

There is virtually no scientific Python software that does not start with `import numpy as np`. So much so that code like the one below would make any experienced programmer cringe. 

<div style="text-align:center">
<img src="./figures/import_hell.png"/>
</div>

[NumPy](https://numpy.org/) (and its close associate [SciPy](https://scipy.org/)) provide with essentially no effort access to all sorts of mathematical functions and utilities, and - if used properly - simplify greatly the task of writing numerical procedures in Python without paying the full performance price of using an interpreted language. 

It is impossible to cover all functionalities of `numpy` in a short module, so we will focus in particular on the use and semantics of NumPy arrays.

## Storage classes in Python

Before delving into the semantics of `numpy.array` objects, let's briefly recap the use of basic storage classes in native Python. You should already know all of this, but just in case...

* Lists are created by writing comma-separated elements between square brackets. They can contain pretty much anything, including elements of different types. They can be updated by adding elements at the end of them using `my_list.append(entry)`, and are characterized by being _ordered_ storage classes, meaning that entries are identified by an unsigned integer between `0` and `len(my_list)-1` and can be accessed as `my_list[position]`. 

```python
my_list = [0, "pizza", 123., Triangle(2,2,2)]
my_list.append("pasta")
my_list[2] = 321.
print(my_list[2])
```

* Tuples are essentially fixed-length, immutable lists. They are created by enumerating elements between round brackets, and the entries can be accessed as `my_tuple[position]`, but cannot be modified

```python
my_tuple = (0, "pizza", 123., Triangle(2,2,2))
print(my_tuple[2])
```

* Dictionaries are associative maps, that link labels (that can be any type which can be [hashed](https://en.wikipedia.org/wiki/Hash_table)) to values, that can also be all sorts of objects and values. They can be created by listing `key:value` pairs between curly brackets, and accessed by the key value. Setting a missing element will create an entry in the dictionary.

```python
my_dict = { "hello": "world", 1234: "is", "a triangle": Triangle(1,1,1) }
my_dict[15] = "fifteen"
print(my_dict["hello"])


## NumPy arrays: creation and indexing

NumPy arrays behave very much like vectors and matrices. Even though in principle they can be created as heterogeneous lists, they are often used to store homogeneous lists of integers or floating point numbers. Contrary to lists, `append` does not add entries "in place" but creates a new array with the element(s) appended. 

NumPy arrays can be created in many ways: starting from a list of values, or initialized using utility functions such as `numpy.zeros` or `numpy.ones`. 
An array has also a `shape`, that determines how you can index the elements as if the array was a tensor. 

```python
import numpy as np
a = np.array([1,2,3,4,5], dtype=int)     # note you can specify the type of the entries of the array
b = np.zeros(shape=(2,4), dtype=float)   # note you can specify the shape of the array
```

In [25]:
a = np.array([1,2,3,4,5], dtype=np.float) # note you can specify the type of the entries of the array
b = np.zeros(shape=(2,4), dtype=np.double)

<ipython-input-25-747920018ee9>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  a = np.array([1,2,3,4,5], dtype=np.float) # note you can specify the type of the entries of the array


In [18]:
a = (4, "hi", Triangle(1,1,1))

In [19]:
a[0]=2

TypeError: 'tuple' object does not support item assignment

In [17]:
import numpy as pd
import pandas as plt
from matplotlib import pyplot as np